# This is script  to divide data by time and calculate clustering params

In [11]:
import csv
import numpy as np
import pandas as pd
import sys
import datetime as dt
from sklearn.cluster import KMeans

Constants

In [12]:
pactcsv = 'pact.csv'
pinfocsv = 'pinfo.csv'
border_times = [3,7,14,28,56,1000]
def convert(s):
            try:
                return int(s)
            except:
                pass
            try:
                return float(s)
            except:
                # booleans
                if s == 'False':
                    return 0
                elif s=='True':
                    return 1
                # genders
                elif s=='f':
                    return -1
                elif s=='m':
                    return 1
                else:
                    return s

Loading of data about actions

In [13]:
with open (pactcsv,'r') as f:
    reader = csv.reader(f,  delimiter='|')
    names = None
    values2 = []
    for row in reader:
        if names == None:
            names = row
            print names
        else:
            values2.append([convert(x) for x in row])

['id', 'actdate', 'actdate_num', 'actflag', 'payflag', 'pgrflag', 'ml_num', 'transactions', 'quests_end', 'm_quests_end', 'grindnum', 'asknum', 'buynum', 'crystallnum', 'days_from_last_actdate', 'days_from_last_payment']


Division of data by time

In [15]:
seconds_in_day =86400
act_date = names.index('actdate')
min_date = min([x[act_date] for x in values2])
print min_date
min_date = dt.datetime.strptime(min_date, "%Y-%m-%d")
divided_data = []
for days in border_times:
    divided_data.append([x for x in values2 if (dt.datetime.strptime(x[act_date], "%Y-%m-%d")-min_date).total_seconds()<=days*seconds_in_day])

2016-01-01


Preprocessing and output of data

In [17]:
def output(data, names, filename):
    with open(filename,'wb') as f:
        writer = csv.writer(f, delimiter='|',
                                quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(names)
        for ID in data:
            writer.writerow(data[ID])
        f.close()
def preprocess(values2, total_days):
    values2.sort(key = lambda x: x[0]) # sort by ID
    values = dict()
    index = 0
    names =['id','actcount', 'med_ml_num','buynum', 'grind','quests_speed']
    while index < len(values2):
        ID = values2[index][0]
        i = index + 1
        while i< len(values2) and values2[i][0] == ID:
            i +=1
        sl = values2[index:i-1]
        q = len(sl)
        if q==0:
            index+=1
            continue
        try:
            values[ID] = [ID]
            values[ID].append(q)
            def to_float(s):
                try:
                    return float(s)
                except:
                    return 0
            values[ID].append((max([to_float(x[6]) for x in sl])-min([to_float(x[6]) for x in sl]))*1.0/q)# speed of ml_num 
            values[ID].append(sum([to_float(x[12]) for x in sl])*1.0/q) # medium count of buynum
            try:
                values[ID].append(sum([to_float(x[11]) for x in sl])*1.0/q) # medium grind
            except IndexError:
                values[ID].append(0)
            values[ID].append((max([to_float(x[8]) for x in sl])-min([to_float(x[8]) for x in sl]))*1.0/q) # quest speed
        except Exception as err:
            print "ERROR: "+format(err) 
            print ID
            print sl
        index = i
    lens = set()
    for ID in values:
        lens.add(len(values[ID]))
    print lens
    output(values, names, 'preprocessed_%d.csv'%total_days)

for i in xrange(len(border_times)):
    preprocess(divided_data[i],border_times[i])


set([6])
set([6])
set([6])
set([6])
set([6])
set([6])
